# Import thư viện

### Nạp thư viện

In [28]:
pip install -r req.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import thư viện

In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

# Tiền xử lí dữ liệu



### Nạp model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=6)

#### lưu model (optional)

In [ ]:
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

### Nạp dataset

In [24]:
# Tải dữ liệu từ file CSV
dataset = load_dataset("csv", data_files={"train": "dataset.csv", "test": "dataset.csv"})

# Lấy danh sách các nhãn gốc từ cột 'intent'
intent_labels = dataset["train"]["intent"]  # Cột 'intent' chứa các nhãn gốc

# Khởi tạo LabelEncoder để mã hóa nhãn thành chỉ số
label_encoder = LabelEncoder()

# Mã hóa các nhãn gốc thành các chỉ số
label_encoder.fit(intent_labels)  # Fit trên toàn bộ nhãn từ cột 'intent'

# In ra các nhãn gốc và các chỉ số đã mã hóa
print("Danh sách nhãn gốc:", label_encoder.classes_)  # In mảng nhãn gốc
print("Các nhãn đã mã hóa:", label_encoder.transform(intent_labels))  # In nhãn đã mã hóa thành chỉ số

# Mã hóa nhãn intent thành chỉ số và tokenize văn bản
def encode_labels_and_tokenize(examples):
    # Tokenize văn bản
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=13)
    
    # Mã hóa intent thành nhãn (labels) sử dụng LabelEncoder đã được fit
    tokenized["labels"] = label_encoder.transform(examples["intent"])  # Mã hóa nhãn thành chỉ số
    
    return tokenized

# Áp dụng mã hóa nhãn và tokenization cho cả train và test dataset
train_dataset = dataset["train"].map(encode_labels_and_tokenize, batched=True)
test_dataset = dataset["test"].map(encode_labels_and_tokenize, batched=True)

# Kiểm tra dữ liệu sau khi mã hóa nhãn và tokenization
  
print(train_dataset["labels"])  # Hiển thị mẫu đầu tiên trong train dataset

Danh sách nhãn gốc: ['clear_history' 'find_api' 'find_history' 'find_news' 'log_in' 'log_out']
Các nhãn đã mã hóa: [2 1 5 4 5 4 1 1 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0]
[2, 1, 5, 4, 5, 4, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Finetuning

### Cấu hình tham số huấn luyện

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir='./logs',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


### Finetuning

In [ ]:
trainer.train()

from google.colab import files
files.download('results')